In [2]:
## create trajectories
import pyarrow.csv as pc
import pyarrow as pa

# Define the path to your CSV file
csv_file = "./data/cleaned/1.csv"  # Change this to your actual file path

# Read the CSV file into a PyArrow Table
try:
    table = pc.read_csv(csv_file)
    print("CSV file successfully read into a PyArrow Table.")
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()

# Print column names and their types
schema = table.schema
print("Columns and their types:")
for field in schema:
    print(f"{field.name}: {field.type}")

# Convert to a pandas DataFrame
df = table.to_pandas()
print("Converted to pandas DataFrame.")

# Display first few rows
print(df.head())
print(df.dtypes)

CSV file successfully read into a PyArrow Table.
Columns and their types:
Timestamp: timestamp[ns]
Type of mobile: string
MMSI: int64
Latitude: double
Longitude: double
Navigational status: string
ROT: double
SOG: double
COG: double
Heading: int64
IMO: int64
Callsign: string
Name: string
Ship type: string
Cargo type: string
Width: int64
Length: int64
Type of position fixing device: string
Draught: double
Destination: string
ETA: timestamp[s]
Data source type: string
A: int64
B: int64
C: int64
D: int64
__index_level_0__: int64
__index_level_0__: int64
Converted to pandas DataFrame.
   Timestamp Type of mobile       MMSI   Latitude  Longitude  \
0 2025-01-26        Class A  245241000  55.398643  14.711777   
1 2025-01-26        Class A  245241000  55.398643  14.711777   
2 2025-01-26        Class A  256883000  54.743528  16.130240   
3 2025-01-26        Class A  636020259  55.053283  14.032850   
4 2025-01-26        Class A  215662000  55.781642  15.883830   

      Navigational status  

In [3]:
## Modelling
df.shape
# -> Turn 9 MIL ais pos into trajectories/trips



(9189296, 28)

In [4]:
import numpy as np
import pandas as pd
from datetime import timedelta

# --- 1. Helper Functions ---

def haversine(lon1, lat1, lon2, lat2):
    """
    Compute the great-circle distance between two points (in km) on Earth.
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371  # Radius of Earth in kilometers.
    return c * r

def compute_ship_features(L, W, A):
    """
    Compute geometric features for a ship.
    
    Parameters:
      L (float): Ship's length.
      W (float): Ship's width (beam).
      A (float): Distance from the ship's bow (or another reference point) to the bridge/antenna.
      
    Returns:
      dict: A dictionary containing computed features.
    """
    Ps = 2 * (L + W)              # Naive Perimeter
    As = L * W                    # Naive Area
    AR = W / L if L != 0 else np.nan  # Aspect Ratio
    Cs = ((L + W) ** 2) / (L * W) if L * W != 0 else np.nan  # Shape Complexity
    BP = A / L if L != 0 else np.nan   # Bridge Position Ratio
    return {
        "Naive_Perimeter": Ps,
        "Naive_Area": As,
        "Aspect_Ratio": AR,
        "Shape_Complexity": Cs,
        "Bridge_Position_Ratio": BP
    }

# --- 2. Segmenting the Data into Trips ---

# Make sure data is sorted by vessel and timestamp
df = df.sort_values(["MMSI", "Timestamp"]).reset_index(drop=True)

# Compute time differences (in seconds) within each vessel group
df["time_diff"] = df.groupby("MMSI")["Timestamp"].diff().dt.total_seconds()

# Define a gap threshold (e.g., 1 hour = 3600 seconds) to mark a new trip.
trip_gap = 3600  
df["new_trip"] = (df["time_diff"] > trip_gap) | (df["time_diff"].isna())

# Create a trip ID by cumulatively summing the "new_trip" flag within each vessel.
df["trip_id"] = df.groupby("MMSI")["new_trip"].cumsum()

# --- 3. Compute Trip-Level Features ---

def compute_trip_features(trip):
    """ 
    Compute aggregated features for one trip (i.e. one group of AIS messages).
    """
    # Ensure the trip is sorted in time
    trip = trip.sort_values("Timestamp")
    
    # Basic trip info
    trip_start = trip["Timestamp"].iloc[0]
    trip_end = trip["Timestamp"].iloc[-1]
    trip_duration_sec = (trip_end - trip_start).total_seconds()
    num_positions = len(trip)
    
    # Trajectory (using haversine distance between consecutive points)
    lats = trip["Latitude"].values
    lons = trip["Longitude"].values
    # Compute distance between each consecutive point
    distances = haversine(lons[:-1], lats[:-1], lons[1:], lats[1:])
    trajectory_length_km = distances.sum()
    
    # Distance between endpoints
    endpoint_distance_km = haversine(lons[0], lats[0], lons[-1], lats[-1])
    
    # Directness ratio
    directness_ratio = trajectory_length_km / endpoint_distance_km if endpoint_distance_km > 0 else np.nan
    
    # Bounding box features
    min_lat = trip["Latitude"].min()
    max_lat = trip["Latitude"].max()
    min_lon = trip["Longitude"].min()
    max_lon = trip["Longitude"].max()
    lat_span = max_lat - min_lat
    lon_span = max_lon - min_lon
    
    # SOG (Speed Over Ground) statistics
    sog_min = trip["SOG"].min()
    sog_max = trip["SOG"].max()
    sog_mean = trip["SOG"].mean()
    sog_median = trip["SOG"].median()
    sog_std = trip["SOG"].std()
    
    # COG (Course Over Ground) statistics
    cog_min = trip["COG"].min()
    cog_max = trip["COG"].max()
    cog_mean = trip["COG"].mean()
    cog_median = trip["COG"].median()
    cog_std = trip["COG"].std()
    
    # Total absolute change in COG, handling wrap-around at 360 degrees.
    cog_values = trip["COG"].values
    # Compute consecutive differences
    cog_diffs = np.abs(np.diff(cog_values))
    # Adjust differences greater than 180 degrees (e.g., 350 -> 10 should be 20, not 340)
    cog_diffs = np.where(cog_diffs > 180, 360 - cog_diffs, cog_diffs)
    total_abs_cog_change = np.sum(cog_diffs)
    
    # Initial heading (from first two positions) expressed as cosine and sine.
    if num_positions >= 2:
        delta_lat = lats[1] - lats[0]
        delta_lon = lons[1] - lons[0]
        # Note: In a more rigorous treatment you may wish to compute the bearing properly,
        # but here we simply compute an angle (in radians) for demonstration.
        init_angle = np.arctan2(delta_lat, delta_lon)
        init_cos = np.cos(init_angle)
        init_sin = np.sin(init_angle)
    else:
        init_cos, init_sin = np.nan, np.nan
    
    # Get static ship information from the first row of the trip.
    static = trip.iloc[0]
    # Ship dimensions and reference value for geometric features.
    L = static["Length"]
    W = static["Width"]
    A_val = static["A"]
    ship_feats = compute_ship_features(L, W, A_val)
    
    # Collect all features in a dictionary.
    features = {
        "trip_start": trip_start,
        "trip_end": trip_end,
        "trip_duration_sec": trip_duration_sec,
        "num_positions": num_positions,
        "trajectory_length_km": trajectory_length_km,
        "endpoint_distance_km": endpoint_distance_km,
        "directness_ratio": directness_ratio,
        "min_lat": min_lat,
        "max_lat": max_lat,
        "min_lon": min_lon,
        "max_lon": max_lon,
        "lat_span": lat_span,
        "lon_span": lon_span,
        "sog_min": sog_min,
        "sog_max": sog_max,
        "sog_mean": sog_mean,
        "sog_median": sog_median,
        "sog_std": sog_std,
        "cog_min": cog_min,
        "cog_max": cog_max,
        "cog_mean": cog_mean,
        "cog_median": cog_median,
        "cog_std": cog_std,
        "total_abs_cog_change": total_abs_cog_change,
        "init_cos": init_cos,
        "init_sin": init_sin
    }
    
    # Add the computed ship (static) features.
    features.update(ship_feats)
    
    # Optionally add other static fields (assuming they do not change during the trip)
    for field in ["Type of mobile", "Navigational status", "Ship type", "Cargo type", "Callsign", "Name", "Destination"]:
        features[field] = static[field]
    
    return pd.Series(features)

# --- 4. Group by Vessel and Trip and Aggregate ---

# Group by MMSI and our computed trip_id and apply our feature extraction.
trip_features_df = df.groupby(["MMSI", "trip_id"]).apply(compute_trip_features).reset_index()

# Inspect the resulting trip-level features:
print(trip_features_df.head())


        MMSI  trip_id          trip_start            trip_end  \
0  111219518        1 2025-01-23 11:54:42 2025-01-23 13:35:50   
1  205482000        1 2025-01-29 19:26:21 2025-01-29 22:32:32   
2  205482000        2 2025-01-29 23:33:52 2025-01-30 00:52:21   
3  209014000        1 2025-01-24 15:54:35 2025-01-24 19:36:45   
4  209014000        2 2025-01-24 20:42:35 2025-01-24 23:25:44   

   trip_duration_sec  num_positions  trajectory_length_km  \
0             6068.0           4011            245.365185   
1            11171.0           1186             60.081535   
2             4709.0              9             24.959654   
3            13330.0           1865             53.123009   
4             9789.0            172             42.907039   

   endpoint_distance_km  directness_ratio    min_lat  ...  Aspect_Ratio  \
0              2.778663         88.303315  55.086065  ...      0.250000   
1             58.911608          1.019859  54.948877  ...      0.144444   
2             24.

/tmp/ipykernel_965735/414938859.py:181: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trip_features_df = df.groupby(["MMSI", "trip_id"]).apply(compute_trip_features).reset_index()


In [6]:
trip_features_df.shape

(2289, 40)

In [12]:
from geopy.distance import geodesic

def compute_bbox_area(row):
    width = geodesic((row["min_lat"], row["min_lon"]), (row["min_lat"], row["max_lon"])).meters
    height = geodesic((row["min_lat"], row["min_lon"]), (row["max_lat"], row["min_lon"])).meters
    area_km2 = (width * height) / 1e6  # Convert to km²
    return area_km2

# Apply function to each row
trip_features_df["total_km2"] = trip_features_df.apply(compute_bbox_area, axis=1)

In [13]:
# drop "Navigational status" column
# drop rows where num_points less than 50
# drop rows where total_km2 is less than 3
print(trip_features_df.shape, " before")
if "Navigational status" in trip_features_df.columns:  # Check if the column exists before dropping
    trip_features_df = trip_features_df.drop("Navigational status", axis=1)
else:
    print("Column 'Navigational status' not found in DataFrame.")


# Drop rows where num_points is less than 50
trip_features_df = trip_features_df[trip_features_df["num_positions"] >= 50]
print(trip_features_df.shape, " after removing num_positions")

# Drop rows where total_km2 is less than 3
trip_features_df = trip_features_df[trip_features_df["total_km2"] >= 3]
print(trip_features_df.shape, " after removing total_km2")

(2138, 40)  before
Column 'Navigational status' not found in DataFrame.
(2138, 40)  after removing num_positions
(2101, 40)  after removing total_km2


In [18]:
# drop crap
trip_features_df = trip_features_df.drop(columns=['total_abs_cog_change'])

In [19]:
trip_features_df.to_csv("./data/cleaned/2.csv")